# Search engine with Tools and Agents

In [30]:
## Arxiv --Research
## Tools Creation
## ArxivqueryRun,WikipediaQueryRun is needed to interact with both the sites outside of langchain to run a user query
## ArxivAPIWrapper,WikipediaAPIWrapper is needed to allow the third party data source access from langchain framework
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper



In [31]:
## Used the in-built tool of wikipedia 
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [32]:
## Used in-built Arxiv Tool
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv) 
arxiv.name



'arxiv'

In [33]:
## our in-built tools
tools = [wiki,arxiv]


In [34]:
## Creating our custom tools [RAG Tools]

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

embeddings = HuggingFaceEmbeddings(model= "jason1234/arzington_bert_embedding_model_v6")

In [35]:
loader = WebBaseLoader(web_path = "https://docs.smith.langchain.com/")
docs = loader.load()
splits= RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100).split_documents(documents=docs)
vectordb = FAISS.from_documents(splits,embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000191CF9000A0>, search_kwargs={})

In [36]:
## Creating retriever Tool

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search engine tool")
retriever_tool.name


'langsmith-search'

In [37]:
tools=[arxiv,wiki,retriever_tool]

## Run all these Tools & LLMs using Agents

In [38]:
## Using Groq for LLM model here

from langchain_groq import ChatGroq
from langchain_openai import OpenAI
import os
import openai
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(model_name = "llama3-70b-8192", groq_api_key = groq_api_key)


In [39]:
## Alternative way to create prompt by importing a prompt template defined on Hub

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [40]:
## Creating Tools Agent to utilise our list of tools[wiki,arxiv,retriever_tool] that we have created earlier
## It will query the data source provided in web path but if query is from outside of that 
# then it will make use of apt tool that we ahve created
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
## Executing Tools Agent
## It will call the tools[wiki,arxiv] if LLM is not capable of answering the query using given data surce here which is LangSmith website
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent, tools=tools,verbose=True)

agent_executor.invoke({"input" : "Tell me about creating agents using openai."})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'creating agents using openai'}`


Page: ChatGPT
Summary: ChatGPT is a generative artificial intelligence chatbot developed by OpenAI and released on November 30, 2022. It uses large language models (LLMs) such as GPT-4o along with other multimodal models to generate human-like respon
Invoking: `langsmith-search` with `{'query': 'creating agents using openai'}`
responded: It seems like the Wikipedia tool provided some information about ChatGPT, but it doesn't directly answer your question about creating agents using OpenAI. Let me try to help you further.



Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and sc

{'input': 'Tell me about creating agents using openai.',
 'output': "It seems that the tool call yielded information about LangSmith, a platform for building ambient agents. To provide a more direct answer to your original question, I'll respond without using a tool.\n\nCreating agents using OpenAI typically involves using their APIs and models to develop conversational AI systems. OpenAI provides various tools and resources, such as the OpenAI API, LangChain, and LangGraph, to help developers build and customize their agents.\n\nTo get started, you can explore OpenAI's documentation and guides, which provide information on how to use their APIs, models, and frameworks to create agents. You can also experiment with their playgrounds and tutorials to gain hands-on experience.\n\nAdditionally, you can leverage OpenAI's pre-trained models, such as ChatGPT, to develop more advanced agents that can understand and respond to user input in a more human-like way.\n\nIf you have more specific q